In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandasql
import gc
import multiprocessing
import os,sys
import missingno as msno
import gc
import matplotlib.pyplot as plt

In [2]:
%%time
train_metadata = pd.read_csv('train_metadata_final.csv')
test_metadata = pd.read_csv('test_metadata_final.csv')

CPU times: user 1min 17s, sys: 4.18 s, total: 1min 21s
Wall time: 1min 21s


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
def reduce_mem_usage(df):

    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [5]:
%%time
train_metadata = reduce_mem_usage(train_metadata)
test_metadata = reduce_mem_usage(test_metadata)

Memory usage of dataframe is 8.68 MB
Memory usage after optimization is: 2.71 MB
Decreased by 68.8%
Memory usage of dataframe is 3837.40 MB
Memory usage after optimization is: 1322.44 MB
Decreased by 65.5%
CPU times: user 1min 1s, sys: 1min 17s, total: 2min 18s
Wall time: 2min 18s


In [6]:
train_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,NG_count_flux_err,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_count_flux,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_count_flux_err,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_count_flux,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_count_flux_err,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_count_flux,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_count_flux_err,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_count_flux,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_count_flux_err,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_count_flux,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_count_flux_err,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_count_flux,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_count_flux_err,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_count_flux,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected
0,615,349.000000,-61.937500,320.75,-51.75000,1,0.000000,0.000000,0.000000,NaN,0.016998,92,352,2.130859,12.845472,1.744747,1577.925415,4.482743,3.835938,352,-1100.440063,660.626343,394.109863,-43330.144531,-123.097000,-89.500000,352,0.226318,333.0,0.945801,1.0,63,2.844200,4.737393,0.493621,240.877243,3.823448,3.866380,63,-116.913223,125.182808,83.944733,-205.036926,-3.254555,-10.015225,63,0.352783,54,0.856934,1.0,58,3.035156,6.953125,1.167969,291.682800,5.027344,4.980469,58,-1100.000000,660.626343,601.787292,-22370.595703,-385.750000,-488.000000,58,0.184082,56,0.965332,1.0,58,2.130859,4.332031,0.579590,196.0000,3.380859,3.388672,58,-682.000000,611.984558,455.121338,-7780.500977,-134.146561,-265.750000,58,0.131348,57,0.982910,1.0,58,2.500000,4.011719,0.347412,193.726715,3.339844,3.388672,58,-530.500000,445.737061,335.425049,-7024.002930,-121.103500,-162.125000,58,0.131348,57,0.982910,1.0,58,2.900391,11.398438,1.088867,220.328110,3.798828,3.625000,58,-422.184509,381.953735,291.803436,-3245.366455,-55.954594,-103.541367,58,0.131348,57,0.982910,1.0,57,6.074219,12.843750,1.318359,435.253174,7.636719,7.308594,57,-422.815094,378.188141,294.779510,-2704.641357,-47.449848,-85.524307,57,0.285400,52,0.912109,1.0
1,713,53.093750,-27.781250,223.50,-54.46875,1,1.818359,1.626953,0.255127,45.40625,0.007000,88,350,0.639648,9.115748,1.509888,825.867004,2.359620,1.998047,350,-14.735178,14.770886,6.471144,-498.172760,-1.423351,-0.873047,350,0.377441,60.0,0.171387,0.0,70,1.470152,3.348282,0.411563,163.198029,2.331400,2.273303,70,-14.735178,14.509829,7.113509,-190.427856,-2.720398,-3.096805,70,0.352539,10,0.142822,0.0,56,0.707031,3.544922,0.707031,79.351021,1.416992,1.115234,56,-11.718750,9.129021,5.712334,-57.109047,-1.019531,-0.561523,56,0.446777,15,0.267822,0.0,56,0.639648,2.531250,0.439697,66.8125,1.193359,1.062500,56,-10.070

In [7]:
print(train_metadata.shape,test_metadata.shape)

(7848, 145) (3492890, 144)


In [8]:
%%time
train_3sigma = pd.read_csv('train_3sigma.csv')
test_3sigma = pd.read_csv('test_3sigma.csv')

CPU times: user 11.2 s, sys: 693 ms, total: 11.9 s
Wall time: 11.9 s


In [9]:
print(train_3sigma.shape,test_3sigma.shape)

(47088, 10) (20957340, 10)


In [10]:
test_3sigma.shape[0] / test_metadata.shape[0]

6.0

In [11]:
%%time
train0 = train_3sigma[train_3sigma['passband'] == 0]
train1 = train_3sigma[train_3sigma['passband'] == 1]
train2 = train_3sigma[train_3sigma['passband'] == 2]
train3 = train_3sigma[train_3sigma['passband'] == 3]
train4 = train_3sigma[train_3sigma['passband'] == 4]
train5 = train_3sigma[train_3sigma['passband'] == 5]
del train_3sigma
train0 = train0.reset_index(drop=True)
train1 = train1.reset_index(drop=True)
train2 = train2.reset_index(drop=True)
train3 = train3.reset_index(drop=True)
train4 = train4.reset_index(drop=True)
train5 = train5.reset_index(drop=True)
train0.columns = ['object_id','passband'] + ['p' + str(0) + '_' + x for x in list(train0.columns)[2:] ]
train1.columns = ['object_id','passband'] + ['p' + str(1) + '_' + x for x in list(train1.columns)[2:] ]
train2.columns = ['object_id','passband'] + ['p' + str(2) + '_' + x for x in list(train2.columns)[2:] ]
train3.columns = ['object_id','passband'] + ['p' + str(3) + '_' + x for x in list(train3.columns)[2:] ]
train4.columns = ['object_id','passband'] + ['p' + str(4) + '_' + x for x in list(train4.columns)[2:] ]
train5.columns = ['object_id','passband'] + ['p' + str(5) + '_' + x for x in list(train5.columns)[2:] ]
del train0['passband'],train1['passband'],train2['passband'],train3['passband'],train4['passband'],train5['passband']
gc.collect()

CPU times: user 48.5 ms, sys: 0 ns, total: 48.5 ms
Wall time: 47.7 ms


In [12]:
train0.head()

,object_id,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4
0,615,NaN,NaN,16.0,17.0,13.0,17.0,NaN,NaN
1,713,NaN,NaN,13.0,22.0,23.0,11.0,1.0,NaN
2,730,NaN,NaN,14.0,22.0,27.0,8.0,NaN,1.0
3,745,NaN,NaN,6.0,38.0,21.0,2.0,3.0,2.0
4,1124,1.0,1.0,7.0,23.0,22.0,9.0,NaN,NaN


In [13]:
%%time
train_metadata = train_metadata.merge(train0,on='object_id',how='left')
train_metadata = train_metadata.merge(train1,on='object_id',how='left')
train_metadata = train_metadata.merge(train2,on='object_id',how='left')
train_metadata = train_metadata.merge(train3,on='object_id',how='left')
train_metadata = train_metadata.merge(train4,on='object_id',how='left')
train_metadata = train_metadata.merge(train5,on='object_id',how='left')

CPU times: user 214 ms, sys: 870 µs, total: 214 ms
Wall time: 212 ms


In [14]:
train_metadata.shape

(7848, 193)

In [15]:
train_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,NG_count_flux_err,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_count_flux,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_count_flux_err,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_count_flux,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_count_flux_err,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_count_flux,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_count_flux_err,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_count_flux,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_count_flux_err,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_count_flux,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_count_flux_err,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_count_flux,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_count_flux_err,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_count_flux,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4
0,615,349.000000,-61.937500,320.75,-51.75000,1,0.000000,0.000000,0.000000,NaN,0.016998,92,352,2.130859,12.845472,1.744747,1577.925415,4.482743,3.835938,352,-1100.440063,660.626343,394.109863,-43330.144531,-123.097000,-89.500000,352,0.226318,333.0,0.945801,1.0,63,2.844200,4.737393,0.493621,240.877243,3.823448,3.866380,63,-116.913223,125.182808,83.944733,-205.036926,-3.254555,-10.015225,63,0.352783,54,0.856934,1.0,58,3.035156,6.953125,1.167969,291.682800,5.027344,4.980469,58,-1100.000000,660.626343,601.787292,-22370.595703,-385.750000,-488.000000,58,0.184082,56,0.965332,1.0,58,2.130859,4.332031,0.579590,196.0000,3.380859,3.388672,58,-682.000000,611.984558,455.121338,-7780.500977,-134.146561,-265.750000,58,0.131348,57,0.982910,1.0,58,2.500000,4.011719,0.347412,193.726715,3.339844,3.388672,58,-530.500000,445.737061,335.425049,-7024.002930,-121.103500,-162.125000,58,0.131348,57,0.982910,1.0,58,2.900391,11.398438,1.088867,220.328110,3.798828,3.625000,58,-422.184509,381.953735,291.803436,-3

In [16]:
%%time
test0 = test_3sigma[test_3sigma['passband'] == 0]
test1 = test_3sigma[test_3sigma['passband'] == 1]
test2 = test_3sigma[test_3sigma['passband'] == 2]
test3 = test_3sigma[test_3sigma['passband'] == 3]
test4 = test_3sigma[test_3sigma['passband'] == 4]
test5 = test_3sigma[test_3sigma['passband'] == 5]
print("A")
del test_3sigma
gc.collect()
test0 = test0.reset_index(drop=True)
test1 = test1.reset_index(drop=True)
test2 = test2.reset_index(drop=True)
test3 = test3.reset_index(drop=True)
test4 = test4.reset_index(drop=True)
test5 = test5.reset_index(drop=True)
print("B")
test0.columns = ['object_id','passband'] + ['p' + str(0) + '_' + x for x in list(test0.columns)[2:] ]
test1.columns = ['object_id','passband'] + ['p' + str(1) + '_' + x for x in list(test1.columns)[2:] ]
test2.columns = ['object_id','passband'] + ['p' + str(2) + '_' + x for x in list(test2.columns)[2:] ]
test3.columns = ['object_id','passband'] + ['p' + str(3) + '_' + x for x in list(test3.columns)[2:] ]
test4.columns = ['object_id','passband'] + ['p' + str(4) + '_' + x for x in list(test4.columns)[2:] ]
test5.columns = ['object_id','passband'] + ['p' + str(5) + '_' + x for x in list(test5.columns)[2:] ]
del test0['passband'],test1['passband'],test2['passband'],test3['passband'],test4['passband'],test5['passband']
print("C")
gc.collect()

A
B
C
CPU times: user 1.35 s, sys: 95.9 ms, total: 1.45 s
Wall time: 1.45 s


In [17]:
%%time
test_metadata = test_metadata.merge(test0,on='object_id',how='left')
print("A")
test_metadata = test_metadata.merge(test1,on='object_id',how='left')
print("B")
test_metadata = test_metadata.merge(test2,on='object_id',how='left')
print("C")
test_metadata = test_metadata.merge(test3,on='object_id',how='left')
print("D")
test_metadata = test_metadata.merge(test4,on='object_id',how='left')
print("E")
test_metadata = test_metadata.merge(test5,on='object_id',how='left')
print("F")

A
B
C
D
E
F
CPU times: user 18.7 s, sys: 2.09 s, total: 20.8 s
Wall time: 20.8 s


In [18]:
print(test_metadata.shape)

(3492890, 192)


In [19]:
train_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,NG_count_flux_err,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_count_flux,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_count_flux_err,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_count_flux,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_count_flux_err,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_count_flux,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_count_flux_err,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_count_flux,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_count_flux_err,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_count_flux,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_count_flux_err,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_count_flux,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_count_flux_err,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_count_flux,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4
0,615,349.000000,-61.937500,320.75,-51.75000,1,0.000000,0.000000,0.000000,NaN,0.016998,92,352,2.130859,12.845472,1.744747,1577.925415,4.482743,3.835938,352,-1100.440063,660.626343,394.109863,-43330.144531,-123.097000,-89.500000,352,0.226318,333.0,0.945801,1.0,63,2.844200,4.737393,0.493621,240.877243,3.823448,3.866380,63,-116.913223,125.182808,83.944733,-205.036926,-3.254555,-10.015225,63,0.352783,54,0.856934,1.0,58,3.035156,6.953125,1.167969,291.682800,5.027344,4.980469,58,-1100.000000,660.626343,601.787292,-22370.595703,-385.750000,-488.000000,58,0.184082,56,0.965332,1.0,58,2.130859,4.332031,0.579590,196.0000,3.380859,3.388672,58,-682.000000,611.984558,455.121338,-7780.500977,-134.146561,-265.750000,58,0.131348,57,0.982910,1.0,58,2.500000,4.011719,0.347412,193.726715,3.339844,3.388672,58,-530.500000,445.737061,335.425049,-7024.002930,-121.103500,-162.125000,58,0.131348,57,0.982910,1.0,58,2.900391,11.398438,1.088867,220.328110,3.798828,3.625000,58,-422.184509,381.953735,291.803436,-3

In [20]:
list0 = list(train_metadata.columns)[-48:-40]
list1 = list(train_metadata.columns)[-40:-32]
list2 = list(train_metadata.columns)[-32:-24]
list3 = list(train_metadata.columns)[-24:-16]
list4 = list(train_metadata.columns)[-16:-8]
list5 = list(train_metadata.columns)[-8:]

In [21]:
print(list4)

['p4_region_minus_4', 'p4_region_minus_3', 'p4_region_minus_2', 'p4_region_minus_1', 'p4_region_plus_1', 'p4_region_plus_2', 'p4_region_plus_3', 'p4_region_plus_4']


In [22]:
%%time
train_metadata['number_of_0s'] = train_metadata[list0].sum(axis=1)
train_metadata['number_of_1s'] = train_metadata[list1].sum(axis=1)
train_metadata['number_of_2s'] = train_metadata[list2].sum(axis=1)
train_metadata['number_of_3s'] = train_metadata[list3].sum(axis=1)
train_metadata['number_of_4s'] = train_metadata[list4].sum(axis=1)
train_metadata['number_of_5s'] = train_metadata[list5].sum(axis=1)

CPU times: user 20.1 ms, sys: 52 µs, total: 20.2 ms
Wall time: 19.1 ms


In [23]:
%%time
test_metadata['number_of_0s'] = test_metadata[list0].sum(axis=1)
test_metadata['number_of_1s'] = test_metadata[list1].sum(axis=1)
test_metadata['number_of_2s'] = test_metadata[list2].sum(axis=1)
test_metadata['number_of_3s'] = test_metadata[list3].sum(axis=1)
test_metadata['number_of_4s'] = test_metadata[list4].sum(axis=1)
test_metadata['number_of_5s'] = test_metadata[list5].sum(axis=1)

CPU times: user 5.51 s, sys: 5.56 s, total: 11.1 s
Wall time: 11.1 s


In [24]:
print(train_metadata.shape,test_metadata.shape)

(7848, 199) (3492890, 198)


In [25]:
train_metadata['number_of_0s'].head()

0    63.0
1    70.0
2    72.0
3    72.0
4    63.0
Name: number_of_0s, dtype: float64

In [26]:
list(train_metadata.columns)[-6]

'number_of_0s'

In [27]:
print(train_metadata.shape,test_metadata.shape)

(7848, 199) (3492890, 198)


In [28]:
train_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,NG_count_flux_err,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_count_flux,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_count_flux_err,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_count_flux,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_count_flux_err,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_count_flux,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_count_flux_err,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_count_flux,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_count_flux_err,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_count_flux,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_count_flux_err,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_count_flux,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_count_flux_err,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_count_flux,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s
0,615,349.000000,-61.937500,320.75,-51.75000,1,0.000000,0.000000,0.000000,NaN,0.016998,92,352,2.130859,12.845472,1.744747,1577.925415,4.482743,3.835938,352,-1100.440063,660.626343,394.109863,-43330.144531,-123.097000,-89.500000,352,0.226318,333.0,0.945801,1.0,63,2.844200,4.737393,0.493621,240.877243,3.823448,3.866380,63,-116.913223,125.182808,83.944733,-205.036926,-3.254555,-10.015225,63,0.352783,54,0.856934,1.0,58,3.035156,6.953125,1.167969,291.682800,5.027344,4.980469,58,-1100.000000,660.626343,601.787292,-22370.595703,-385.750000,-488.000000,58,0.184082,56,0.965332,1.0,58,2.130859,4.332031,0.579590,196.0000,3.380859,3.388672,58,-682.000000,611.984558,455.121338,-7780.500977,-134.146561,-265.750000,58,0.131348,57,0.982910,1.0,58,2.500000,4.011719,0.347412,193.726715,3.339844,3.388672,58,-530.500000,445.737061,335.425049,-7024.002930,-121.103500,-162.125000,58,0.131348,57,0.982910,1.0,58,2.900391,11.398438

In [29]:
%%time
train_metadata['percent_' + 'p0_region_minus_4'] = train_metadata['p0_region_minus_4'] / train_metadata['number_of_0s']
train_metadata['percent_' + 'p0_region_minus_3'] = train_metadata['p0_region_minus_3'] / train_metadata['number_of_0s']
train_metadata['percent_' + 'p0_region_minus_2'] = train_metadata['p0_region_minus_2'] / train_metadata['number_of_0s']
train_metadata['percent_' + 'p0_region_minus_1'] = train_metadata['p0_region_minus_1'] / train_metadata['number_of_0s']
train_metadata['percent_' + 'p0_region_plus_1'] = train_metadata['p0_region_plus_1'] / train_metadata['number_of_0s']
train_metadata['percent_' + 'p0_region_plus_2'] = train_metadata['p0_region_plus_2'] / train_metadata['number_of_0s']
train_metadata['percent_' + 'p0_region_plus_3'] = train_metadata['p0_region_plus_3'] / train_metadata['number_of_0s']
train_metadata['percent_' + 'p0_region_plus_4'] = train_metadata['p0_region_plus_4'] / train_metadata['number_of_0s']

train_metadata['percent_' + 'p1_region_minus_4'] = train_metadata['p1_region_minus_4'] / train_metadata['number_of_1s']
train_metadata['percent_' + 'p1_region_minus_3'] = train_metadata['p1_region_minus_3'] / train_metadata['number_of_1s']
train_metadata['percent_' + 'p1_region_minus_2'] = train_metadata['p1_region_minus_2'] / train_metadata['number_of_1s']
train_metadata['percent_' + 'p1_region_minus_1'] = train_metadata['p1_region_minus_1'] / train_metadata['number_of_1s']
train_metadata['percent_' + 'p1_region_plus_1'] = train_metadata['p1_region_plus_1'] / train_metadata['number_of_1s']
train_metadata['percent_' + 'p1_region_plus_2'] = train_metadata['p1_region_plus_2'] / train_metadata['number_of_1s']
train_metadata['percent_' + 'p1_region_plus_3'] = train_metadata['p1_region_plus_3'] / train_metadata['number_of_1s']
train_metadata['percent_' + 'p1_region_plus_4'] = train_metadata['p1_region_plus_4'] / train_metadata['number_of_1s']

train_metadata['percent_' + 'p2_region_minus_4'] = train_metadata['p2_region_minus_4'] / train_metadata['number_of_2s']
train_metadata['percent_' + 'p2_region_minus_3'] = train_metadata['p2_region_minus_3'] / train_metadata['number_of_2s']
train_metadata['percent_' + 'p2_region_minus_2'] = train_metadata['p2_region_minus_2'] / train_metadata['number_of_2s']
train_metadata['percent_' + 'p2_region_minus_1'] = train_metadata['p2_region_minus_1'] / train_metadata['number_of_2s']
train_metadata['percent_' + 'p2_region_plus_1'] = train_metadata['p2_region_plus_1'] / train_metadata['number_of_2s']
train_metadata['percent_' + 'p2_region_plus_2'] = train_metadata['p2_region_plus_2'] / train_metadata['number_of_2s']
train_metadata['percent_' + 'p2_region_plus_3'] = train_metadata['p2_region_plus_3'] / train_metadata['number_of_2s']
train_metadata['percent_' + 'p2_region_plus_4'] = train_metadata['p2_region_plus_4'] / train_metadata['number_of_2s']

train_metadata['percent_' + 'p3_region_minus_4'] = train_metadata['p3_region_minus_4'] / train_metadata['number_of_3s']
train_metadata['percent_' + 'p3_region_minus_3'] = train_metadata['p3_region_minus_3'] / train_metadata['number_of_3s']
train_metadata['percent_' + 'p3_region_minus_2'] = train_metadata['p3_region_minus_2'] / train_metadata['number_of_3s']
train_metadata['percent_' + 'p3_region_minus_1'] = train_metadata['p3_region_minus_1'] / train_metadata['number_of_3s']
train_metadata['percent_' + 'p3_region_plus_1'] = train_metadata['p3_region_plus_1'] / train_metadata['number_of_3s']
train_metadata['percent_' + 'p3_region_plus_2'] = train_metadata['p3_region_plus_2'] / train_metadata['number_of_3s']
train_metadata['percent_' + 'p3_region_plus_3'] = train_metadata['p3_region_plus_3'] / train_metadata['number_of_3s']
train_metadata['percent_' + 'p3_region_plus_4'] = train_metadata['p3_region_plus_4'] / train_metadata['number_of_3s']

train_metadata['percent_' + 'p4_region_minus_4'] = train_metadata['p4_region_minus_4'] / train_metadata['number_of_4s']
train_metadata['percent_' + 'p4_region_minus_3'] = train_metadata['p4_region_minus_3'] / train_metadata['number_of_4s']
train_metadata['percent_' + 'p4_region_minus_2'] = train_metadata['p4_region_minus_2'] / train_metadata['number_of_4s']
train_metadata['percent_' + 'p4_region_minus_1'] = train_metadata['p4_region_minus_1'] / train_metadata['number_of_4s']
train_metadata['percent_' + 'p4_region_plus_1'] = train_metadata['p4_region_plus_1'] / train_metadata['number_of_4s']
train_metadata['percent_' + 'p4_region_plus_2'] = train_metadata['p4_region_plus_2'] / train_metadata['number_of_4s']
train_metadata['percent_' + 'p4_region_plus_3'] = train_metadata['p4_region_plus_3'] / train_metadata['number_of_4s']
train_metadata['percent_' + 'p4_region_plus_4'] = train_metadata['p4_region_plus_4'] / train_metadata['number_of_4s']

train_metadata['percent_' + 'p5_region_minus_4'] = train_metadata['p5_region_minus_4'] / train_metadata['number_of_5s']
train_metadata['percent_' + 'p5_region_minus_3'] = train_metadata['p5_region_minus_3'] / train_metadata['number_of_5s']
train_metadata['percent_' + 'p5_region_minus_2'] = train_metadata['p5_region_minus_2'] / train_metadata['number_of_5s']
train_metadata['percent_' + 'p5_region_minus_1'] = train_metadata['p5_region_minus_1'] / train_metadata['number_of_5s']
train_metadata['percent_' + 'p5_region_plus_1'] = train_metadata['p5_region_plus_1'] / train_metadata['number_of_5s']
train_metadata['percent_' + 'p5_region_plus_2'] = train_metadata['p5_region_plus_2'] / train_metadata['number_of_5s']
train_metadata['percent_' + 'p5_region_plus_3'] = train_metadata['p5_region_plus_3'] / train_metadata['number_of_5s']
train_metadata['percent_' + 'p5_region_plus_4'] = train_metadata['p5_region_plus_4'] / train_metadata['number_of_5s']


CPU times: user 26.6 ms, sys: 0 ns, total: 26.6 ms
Wall time: 25.3 ms


In [30]:
%%time
test_metadata['percent_' + 'p0_region_minus_4'] = test_metadata['p0_region_minus_4'] / test_metadata['number_of_0s']
test_metadata['percent_' + 'p0_region_minus_3'] = test_metadata['p0_region_minus_3'] / test_metadata['number_of_0s']
test_metadata['percent_' + 'p0_region_minus_2'] = test_metadata['p0_region_minus_2'] / test_metadata['number_of_0s']
test_metadata['percent_' + 'p0_region_minus_1'] = test_metadata['p0_region_minus_1'] / test_metadata['number_of_0s']
test_metadata['percent_' + 'p0_region_plus_1'] = test_metadata['p0_region_plus_1'] / test_metadata['number_of_0s']
test_metadata['percent_' + 'p0_region_plus_2'] = test_metadata['p0_region_plus_2'] / test_metadata['number_of_0s']
test_metadata['percent_' + 'p0_region_plus_3'] = test_metadata['p0_region_plus_3'] / test_metadata['number_of_0s']
test_metadata['percent_' + 'p0_region_plus_4'] = test_metadata['p0_region_plus_4'] / test_metadata['number_of_0s']

test_metadata['percent_' + 'p1_region_minus_4'] = test_metadata['p1_region_minus_4'] / test_metadata['number_of_1s']
test_metadata['percent_' + 'p1_region_minus_3'] = test_metadata['p1_region_minus_3'] / test_metadata['number_of_1s']
test_metadata['percent_' + 'p1_region_minus_2'] = test_metadata['p1_region_minus_2'] / test_metadata['number_of_1s']
test_metadata['percent_' + 'p1_region_minus_1'] = test_metadata['p1_region_minus_1'] / test_metadata['number_of_1s']
test_metadata['percent_' + 'p1_region_plus_1'] = test_metadata['p1_region_plus_1'] / test_metadata['number_of_1s']
test_metadata['percent_' + 'p1_region_plus_2'] = test_metadata['p1_region_plus_2'] / test_metadata['number_of_1s']
test_metadata['percent_' + 'p1_region_plus_3'] = test_metadata['p1_region_plus_3'] / test_metadata['number_of_1s']
test_metadata['percent_' + 'p1_region_plus_4'] = test_metadata['p1_region_plus_4'] / test_metadata['number_of_1s']

test_metadata['percent_' + 'p2_region_minus_4'] = test_metadata['p2_region_minus_4'] / test_metadata['number_of_2s']
test_metadata['percent_' + 'p2_region_minus_3'] = test_metadata['p2_region_minus_3'] / test_metadata['number_of_2s']
test_metadata['percent_' + 'p2_region_minus_2'] = test_metadata['p2_region_minus_2'] / test_metadata['number_of_2s']
test_metadata['percent_' + 'p2_region_minus_1'] = test_metadata['p2_region_minus_1'] / test_metadata['number_of_2s']
test_metadata['percent_' + 'p2_region_plus_1'] = test_metadata['p2_region_plus_1'] / test_metadata['number_of_2s']
test_metadata['percent_' + 'p2_region_plus_2'] = test_metadata['p2_region_plus_2'] / test_metadata['number_of_2s']
test_metadata['percent_' + 'p2_region_plus_3'] = test_metadata['p2_region_plus_3'] / test_metadata['number_of_2s']
test_metadata['percent_' + 'p2_region_plus_4'] = test_metadata['p2_region_plus_4'] / test_metadata['number_of_2s']

test_metadata['percent_' + 'p3_region_minus_4'] = test_metadata['p3_region_minus_4'] / test_metadata['number_of_3s']
test_metadata['percent_' + 'p3_region_minus_3'] = test_metadata['p3_region_minus_3'] / test_metadata['number_of_3s']
test_metadata['percent_' + 'p3_region_minus_2'] = test_metadata['p3_region_minus_2'] / test_metadata['number_of_3s']
test_metadata['percent_' + 'p3_region_minus_1'] = test_metadata['p3_region_minus_1'] / test_metadata['number_of_3s']
test_metadata['percent_' + 'p3_region_plus_1'] = test_metadata['p3_region_plus_1'] / test_metadata['number_of_3s']
test_metadata['percent_' + 'p3_region_plus_2'] = test_metadata['p3_region_plus_2'] / test_metadata['number_of_3s']
test_metadata['percent_' + 'p3_region_plus_3'] = test_metadata['p3_region_plus_3'] / test_metadata['number_of_3s']
test_metadata['percent_' + 'p3_region_plus_4'] = test_metadata['p3_region_plus_4'] / test_metadata['number_of_3s']

test_metadata['percent_' + 'p4_region_minus_4'] = test_metadata['p4_region_minus_4'] / test_metadata['number_of_4s']
test_metadata['percent_' + 'p4_region_minus_3'] = test_metadata['p4_region_minus_3'] / test_metadata['number_of_4s']
test_metadata['percent_' + 'p4_region_minus_2'] = test_metadata['p4_region_minus_2'] / test_metadata['number_of_4s']
test_metadata['percent_' + 'p4_region_minus_1'] = test_metadata['p4_region_minus_1'] / test_metadata['number_of_4s']
test_metadata['percent_' + 'p4_region_plus_1'] = test_metadata['p4_region_plus_1'] / test_metadata['number_of_4s']
test_metadata['percent_' + 'p4_region_plus_2'] = test_metadata['p4_region_plus_2'] / test_metadata['number_of_4s']
test_metadata['percent_' + 'p4_region_plus_3'] = test_metadata['p4_region_plus_3'] / test_metadata['number_of_4s']
test_metadata['percent_' + 'p4_region_plus_4'] = test_metadata['p4_region_plus_4'] / test_metadata['number_of_4s']

test_metadata['percent_' + 'p5_region_minus_4'] = test_metadata['p5_region_minus_4'] / test_metadata['number_of_5s']
test_metadata['percent_' + 'p5_region_minus_3'] = test_metadata['p5_region_minus_3'] / test_metadata['number_of_5s']
test_metadata['percent_' + 'p5_region_minus_2'] = test_metadata['p5_region_minus_2'] / test_metadata['number_of_5s']
test_metadata['percent_' + 'p5_region_minus_1'] = test_metadata['p5_region_minus_1'] / test_metadata['number_of_5s']
test_metadata['percent_' + 'p5_region_plus_1'] = test_metadata['p5_region_plus_1'] / test_metadata['number_of_5s']
test_metadata['percent_' + 'p5_region_plus_2'] = test_metadata['p5_region_plus_2'] / test_metadata['number_of_5s']
test_metadata['percent_' + 'p5_region_plus_3'] = test_metadata['p5_region_plus_3'] / test_metadata['number_of_5s']
test_metadata['percent_' + 'p5_region_plus_4'] = test_metadata['p5_region_plus_4'] / test_metadata['number_of_5s']

CPU times: user 419 ms, sys: 0 ns, total: 419 ms
Wall time: 418 ms


In [37]:
print(train_metadata.shape,test_metadata.shape)

(7848, 247) (3492890, 246)


In [34]:
train_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,NG_count_flux_err,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_count_flux,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_count_flux_err,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_count_flux,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_count_flux_err,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_count_flux,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_count_flux_err,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_count_flux,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_count_flux_err,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_count_flux,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_count_flux_err,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_count_flux,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_count_flux_err,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_count_flux,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,

In [39]:
deleted_list = [x + '_' + 'count' + '_' + y for x in ['NG','A0','A1','A2','A3','A4','A5'] for y in ['flux','flux_err']]

In [40]:
print(deleted_list)

['NG_count_flux', 'NG_count_flux_err', 'A0_count_flux', 'A0_count_flux_err', 'A1_count_flux', 'A1_count_flux_err', 'A2_count_flux', 'A2_count_flux_err', 'A3_count_flux', 'A3_count_flux_err', 'A4_count_flux', 'A4_count_flux_err', 'A5_count_flux', 'A5_count_flux_err']


In [41]:
%%time
train_metadata.drop(deleted_list,axis=1,inplace=True)
test_metadata.drop(deleted_list,axis=1,inplace=True)

CPU times: user 2.3 s, sys: 3.01 s, total: 5.31 s
Wall time: 5.31 s


In [42]:
print(train_metadata.shape,test_metadata.shape)

(7848, 233) (3492890, 232)


In [43]:
train_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_pl

In [44]:
%%time
train_metadata = reduce_mem_usage(train_metadata)
test_metadata = reduce_mem_usage(test_metadata)

Memory usage of dataframe is 8.76 MB
Memory usage after optimization is: 4.19 MB
Decreased by 52.1%
Memory usage of dataframe is 3973.98 MB
Memory usage after optimization is: 1935.36 MB
Decreased by 51.3%
CPU times: user 45.5 s, sys: 43.3 s, total: 1min 28s
Wall time: 1min 28s


In [45]:
%%time
train_metadata.to_csv('train_metadata_final.csv',index=False)
test_metadata.to_csv('test_metadata_final.csv',index=False)

CPU times: user 11min 27s, sys: 11.6 s, total: 11min 38s
Wall time: 11min 40s
